In [10]:
import json
import requests
from datetime import datetime as dt
import hmac
import hashlib
import base64
from biscoint_api_python import Biscoint
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
# Constants
BASE_URL="https://api.biscoint.io/v1"
API_KEY = os.getenv('API_KEY') 
API_SECRET= os.getenv('API_SECRET') 
AMOUNT = '0.00001'

In [237]:
bsc = Biscoint(API_KEY, API_SECRET)
endpoints_meta = bsc.get_meta()

In [289]:
endpoints_meta

{'version': 'v1',
 'endpoints': {'ticker': {'get': {'type': 'public',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 12,
     'rate': '12 per 1 minute'}}},
  'fees': {'get': {'type': 'public',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 2,
     'rate': '2 per 1 minute'}}},
  'meta': {'get': {'type': 'public',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 2,
     'rate': '2 per 1 minute'}}},
  'balance': {'post': {'type': 'private',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 20,
     'rate': '20 per 1 minute'}}},
  'offer': {'post': {'type': 'private',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 30,
     'rate': '30 per 1 minute'}}},
  'offer/confirm': {'post': {'type': 'private',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 30,
     'rate': '30 per 1 minute'}}},
  'trades': {'post': {'type': 'private',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 20,
     'rate': '20 per 1 minute'}}}}}

In [272]:
# Util functions
def sign(endpoint,nonce,data=''):
  param_string = f"v1/{endpoint}{nonce}{data}"
  encoded_param_string = param_string.encode('utf-8')
  base64_param_string = base64.b64encode(encoded_param_string)
  signature = hmac.new(API_SECRET.encode(), base64_param_string, hashlib.sha384).hexdigest()
  return signature

# Generate a headers
def headers(endpoint,data={}):
    nonce_now = nonce()
    return {
        "Content-Type": "application/json",
        "BSCNT-NONCE": nonce_now,
        "BSCNT-APIKEY": API_KEY,
        "BSCNT-SIGN": sign(endpoint,nonce_now,data),
    }


# Generate a nonce
nonce = lambda  : str(int(dt.now().timestamp() * 1000))
# Calulate the percent
percent = lambda   value1, value2  :  round((float(value2) / float(value1) - 1) * 100,3)



In [247]:
headers('balance')

{'Content-Type': 'application/json',
 'BSCNT-NONCE': '1618674569998',
 'BSCNT-APIKEY': 'f37aea67e915bed1fcb837a3cd858ff2c38cf6a00e242b87d0703c9f1a04e010',
 'BSCNT-SIGN': '67e34644cdc55b1550aef20b5a9f70f8a41aa24a898b9241783f099912f4bdf61c0b756e8da45f1d514faddb18a0cf4d'}

In [240]:
#requests.get(f"{BASE_URL}/fees").json()
bsc.get_fees()

{'withdrawal': {'BTC': {'rate': '0.0',
   'fixed': {'slow': '0.00046', 'normal': '0.00052', 'fast': '0.00056'}},
  'BRL': {'rate': '0.0',
   'fixed': {'ted': '14.90', 'sameBankTransfer': '14.90'}}}}

In [241]:
bsc.get_balance()

{'BRL': '0.00', 'BTC': '0.00027316'}

In [242]:
endpoints_meta

{'version': 'v1',
 'endpoints': {'ticker': {'get': {'type': 'public',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 12,
     'rate': '12 per 1 minute'}}},
  'fees': {'get': {'type': 'public',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 2,
     'rate': '2 per 1 minute'}}},
  'meta': {'get': {'type': 'public',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 2,
     'rate': '2 per 1 minute'}}},
  'balance': {'post': {'type': 'private',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 20,
     'rate': '20 per 1 minute'}}},
  'offer': {'post': {'type': 'private',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 30,
     'rate': '30 per 1 minute'}}},
  'offer/confirm': {'post': {'type': 'private',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 30,
     'rate': '30 per 1 minute'}}},
  'trades': {'post': {'type': 'private',
    'rateLimit': {'windowMs': 60000,
     'maxRequests': 20,
     'rate': '20 per 1 minute'}}}}}

In [288]:
buy = bsc.get_offer(op='buy',amount=AMOUNT,isQuote=False)
sell = bsc.get_offer(op='sell',amount=AMOUNT,isQuote=False)
calculated_percent = percent(buy['efPrice'], sell['efPrice'])
print(f"Percent is {calculated_percent}")

if( calculated_percent > 0):
    print("Arbitrage oportunity!")

Percent is -0.583


In [281]:
buy

{'offerId': '3xMmMjW8ehjxu7wHZ',
 'base': 'BTC',
 'quote': 'BRL',
 'op': 'buy',
 'isQuote': False,
 'baseAmount': '0.00001000',
 'quoteAmount': '3.43',
 'efPrice': '343000.00',
 'createdAt': '2021-04-17T16:01:44.676Z',
 'expiresAt': '2021-04-17T16:01:59.676Z',
 'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}

In [284]:
sell

{'offerId': '5s8Te7YPG4bMnMixb',
 'base': 'BTC',
 'quote': 'BRL',
 'op': 'sell',
 'isQuote': False,
 'baseAmount': '0.00001000',
 'quoteAmount': '3.42',
 'efPrice': '342000.00',
 'createdAt': '2021-04-17T16:01:46.932Z',
 'expiresAt': '2021-04-17T16:02:01.932Z',
 'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}

-0.581

In [285]:
r = requests.post(f'{BASE_URL}/balance',headers=headers('balance'))#.json()
r.json()

{'code': 4003,
 'error': 'Forbidden',
 'details': 'Invalid authorization headers'}

In [255]:
r.request.headers

{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'BSCNT-NONCE': '1618674613242', 'BSCNT-APIKEY': 'f37aea67e915bed1fcb837a3cd858ff2c38cf6a00e242b87d0703c9f1a04e010', 'BSCNT-SIGN': 'f1ed9aca5efeaf5b920eb5090d898217e652ff64081ae14cb43377c4980f26eba17cbcab8419b8a895452b72eea83a16', 'Content-Length': '0'}